# Final Dataset Preparation

This notebook exports the final cleaned dataset with and without identification columns.


## Library imports


In [22]:
import pandas as pd 
import joblib
import importlib 
import subprocess
import sys

from pathlib import Path 

In [23]:
# import other libraries 
from helper import rpkl

# check if jcds library is installed
package_name = "jcds"

if importlib.util.find_spec(package_name) is None:
    print(f" '{package_name}' not found. Installing from Github... ")
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "git+https://github.com/junclemente/jcds.git",
        ]
    )
else:
    print(f" '{package_name}' is already installed.")

from jcds import eda as jeda
from jcds import reports as jrep

 'jcds' is already installed.


## Import dataset


In [24]:
data_folder = Path("../data")
raw_pickle = Path( data_folder / "raw_pickle")

df = pd.read_pickle(data_folder / "05_raw_combined_with_ids.pkl")
df["cdscode"] = df["cdscode"].astype(str)

df.head()

,cdscode,county,virtual,magnet,yearroundyn,latitude,longitude,multilingual,cohortstudents,regular_hs_diploma_graduates_rate,...,pct_senior_cohort,pct_unsafe_gr11,pct_safe_gr11,pct_neutral_gr11,avg_safety_score,high_conn,low_conn,conn_ratio,school_climate_index,target_grad_category
0,01611190130229,Alameda,N,N,N,37.764958,-122.24593,N,394.0,92.4,...,0.498894,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115,Graduated / On Track
1,01611270130450,Alameda,N,N,N,37.896661,-122.29257,N,284.0,95.1,...,0.501264,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115,Graduated / On Track
2,01611430131177,Alameda,N,N,N,37.868913,-122.27120,Y,861.0,90.5,...,0.496276,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115,Graduated / On Track
3,01611500132225,Alameda,N,N,N,37.705184,-122.07847,N,672.0,96.4,...,0.470174,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115,Graduated / On Track
5,01611680132746,Alameda,N,N,N,37.835405,-122.28270,N,40.0,95.0,...,0.545455,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115,Graduated / On Track


# Create final dataset for modeling

## Features to drop

- `dropout_rate` - secondary target, strong correlation with target variable
- `avg_safety_score` - perfect correlation with `school_climate_index`
- `pct_inexperienced` - correlated with `pct_experienced`
- `eligible_cumulative_enrollment` - correlated with `cohortstudents`
- `pct_unsafe_gr11`, `pct_safe_gr11`, `pct_neutral_gr11` - missing >16% data, correlation with `avg_safety_score`
- `pct_first_year` - correlated with `pct_experienced`
- `high_conn`, `low_conn`, `conn_ratio` - low variance and therefore no predictive power

All categorical variables have low variance and therefore have low predictive power. These will also be dropped from the dataset.

- `virtual`
- `magnet`
- `yearroundyn`
- `multilingual`

The following features that are only used for identification.

- `county`
- `cdscode`
- `latitude`
- `longitude`


## Dataset with safety climate imputed


In [25]:
# columns to drop
features_to_drop = [
    'dropout_rate',
    'avg_safety_score',
    'pct_inexperienced', 
    'eligible_cumulative_enrollment', 
    'pct_unsafe_gr11', 
    'pct_safe_gr11', 
    'pct_neutral_gr11', 
    'pct_first_year',
    'high_conn', 
    'low_conn', 
    'conn_ratio',
    'virtual',
    'magnet', 
    'yearroundyn', 
    'multilingual',
    'county', 
    'latitude', 
    'longitude'
]

id_column = [
    'cdscode'
]


In [26]:
print(f"Shape before dropping features: {df.shape}")
df = df.drop(columns=features_to_drop)
print(f"Shape after dropping features: {df.shape}")

Shape before dropping features: (958, 46)
Shape after dropping features: (958, 28)


### Imputation of missing data

For missing data in each feature, median imputation will be performed.


In [27]:
# check missing and value being imputed
missing_before = df.isnull().sum()
imputed_medians = df.median(numeric_only=True)

for col, missing_count in missing_before.items():
    if missing_count > 0:
        print(f"{col}: {missing_count} values imputed with median ({imputed_medians[col]:.2f})")

stu_tch_ratio: 14 values imputed with median (22.00)
stu_adm_ratio: 58 values imputed with median (414.00)
stu_psv_ratio: 62 values imputed with median (274.70)
pct_associate: 14 values imputed with median (0.00)
pct_bachelors: 14 values imputed with median (0.17)
pct_bachelors_plus: 14 values imputed with median (0.26)
pct_master: 14 values imputed with median (0.34)
pct_master_plus: 14 values imputed with median (0.08)
pct_doctorate: 14 values imputed with median (0.01)
pct_juris_doctor: 14 values imputed with median (0.00)
pct_no_degree: 14 values imputed with median (0.00)
pct_experienced: 14 values imputed with median (0.89)
pct_second_year: 14 values imputed with median (0.04)
grade_retention_ratio: 2 values imputed with median (0.92)
school_climate_index: 106 values imputed with median (0.52)


In [28]:
# impute missing values
df = df.fillna(df.median(numeric_only=True))

In [29]:
jrep.data_info(df, show_columns=True)


SHAPE:
There are 958 rows and 28 columns (0.33 MB).

DUPLICATES:
There are 0 duplicated rows.

COLUMNS/VARIABLES:
Column dType Summary:
 * object: 2
 * float: 26
There are 26 numerical (int/float/bool) variables.
 * Columns: ['cohortstudents', 'regular_hs_diploma_graduates_rate', 'met_uccsu_grad_reqs_rate', 'seal_of_biliteracy_rate', 'still_enrolled_rate', 'chronicabsenteeismrate', 'unexcused_absences_percent', 'outofschool_suspension_absences_percent', 'percent__eligible_free_k12', 'frpm_count_k12', 'stu_tch_ratio', 'stu_adm_ratio', 'stu_psv_ratio', 'pct_associate', 'pct_bachelors', 'pct_bachelors_plus', 'pct_master', 'pct_master_plus', 'pct_doctorate', 'pct_juris_doctor', 'pct_no_degree', 'pct_experienced', 'pct_second_year', 'grade_retention_ratio', 'pct_senior_cohort', 'school_climate_index']
There are 2 categorical (nominal/ordinal) variables.
 * Columns: ['cdscode', 'target_grad_category']

DATETIME COLUMNS:
There are 0 datetime variables and 0 possible datetime variables.

OTHE

In [30]:
jrep.data_cardinality(df, show_columns=True)

CARDINALITY REPORT

Total columns analyzed: 28

[BINARY COLUMNS]
There are 1 binary columns.
 * Columns: ['target_grad_category']
There are 0 binary with nan.

[CONSTANT/NEAR CONSTANT COLUMNS]
There are 0 constant columns.
There are 0 near-constant columns with >= 95% of values being the same.

[LOW CARDINALITY CATEGORICAL COLUMNS]
 * There are 1 low cardinality columns with <= 10 unique values.
Columns:
 * target_grad_category: 2 unique values

[HIGH CARDINALITY CATEGORICAL COLUMNS]
 * There are 1 high cardinality variables with >=90% unique values.
 * Columns: [('cdscode', 100.0)]


In [31]:
jrep.data_quality(df, show_columns=True)

DATA QUALITY REPORT

 * Total entries (rows * cols): 26824
 * Memory usage: 0.33 MB
 * Rows: 958
 * Columns: 28

MISSING DATA:
 * Total entries: 0 missing (0.0%)

ROWS:
----------
 * Rows missing any: 0
 * Rows missing all: 0

DUPLICATES: 0

COLUMNS:
----------------
Columns missing any: 0

CONSTANT: 0

NEAR CONSTANT: 0
	(95% of values are the same)

MIXED DATATYPES: 0

HIGH CARDINALITY: 1
	(60% >= unique values)
	* 'cdscode': 100.0%


### Rename target variables


In [32]:
target = "regular_hs_diploma_graduates_rate"

# rename numerical target variable
df = df.rename(columns = {target: 'graduation_rate'})

# rename categorical target variable
df = df.rename(columns = {'target_grad_category': 'high_grad_rate'})

## Dataset: Drop safety, keep all counties


In [33]:
df = df.drop(columns=["school_climate_index"])
df.head()


,cdscode,cohortstudents,graduation_rate,met_uccsu_grad_reqs_rate,seal_of_biliteracy_rate,still_enrolled_rate,chronicabsenteeismrate,unexcused_absences_percent,outofschool_suspension_absences_percent,percent__eligible_free_k12,...,pct_master,pct_master_plus,pct_doctorate,pct_juris_doctor,pct_no_degree,pct_experienced,pct_second_year,grade_retention_ratio,pct_senior_cohort,high_grad_rate
0,01611190130229,394.0,92.4,73.9,22.8,1.0,12.7,23.5,0.5,0.172013,...,0.126316,0.347368,0.021053,0.0,0.063158,0.863158,0.031579,1.058568,0.498894,Graduated / On Track
1,01611270130450,284.0,95.1,67.8,21.5,0.0,70.3,46.2,0.4,0.174389,...,0.342236,0.079646,0.011065,0.0,0.000000,0.894523,0.038462,0.996721,0.501264,Graduated / On Track
2,01611430131177,861.0,90.5,62.3,12.1,0.8,5.2,24.1,0.0,0.262259,...,0.360169,0.046610,0.016949,0.0,0.000000,0.855932,0.050847,1.014907,0.496276,Graduated / On Track
3,01611500132225,672.0,96.4,72.8,25.0,0.0,3.5,28.0,0.9,0.166358,...,0.161074,0.315436,0.020134,0.0,0.000000,0.912752,0.033557,0.829876,0.470174,Graduated / On Track
5,01611680132746,40.0,95.0,76.3,0.0,0.0,29.6,67.9,0.0,0.551136,...,0.333333,0.166667,0.000000,0.0,0.000000,1.000000,0.000000,1.170732,0.545455,Graduated / On Track


### Create target column


In [34]:
# verify low_grad_rate exists or create
if "low_grad_rate" not in df.columns:
    df["low_grad_rate"] = (df["graduation_rate"] < 90).astype(int)

# drop redundant columns 
redundant_cols = [
    "high_grad_rate"       
]

df = df.drop(columns=redundant_cols, errors = "ignore")

print("Shape:", df.shape[1])
df.head()

Shape: 27


,cdscode,cohortstudents,graduation_rate,met_uccsu_grad_reqs_rate,seal_of_biliteracy_rate,still_enrolled_rate,chronicabsenteeismrate,unexcused_absences_percent,outofschool_suspension_absences_percent,percent__eligible_free_k12,...,pct_master,pct_master_plus,pct_doctorate,pct_juris_doctor,pct_no_degree,pct_experienced,pct_second_year,grade_retention_ratio,pct_senior_cohort,low_grad_rate
0,01611190130229,394.0,92.4,73.9,22.8,1.0,12.7,23.5,0.5,0.172013,...,0.126316,0.347368,0.021053,0.0,0.063158,0.863158,0.031579,1.058568,0.498894,0
1,01611270130450,284.0,95.1,67.8,21.5,0.0,70.3,46.2,0.4,0.174389,...,0.342236,0.079646,0.011065,0.0,0.000000,0.894523,0.038462,0.996721,0.501264,0
2,01611430131177,861.0,90.5,62.3,12.1,0.8,5.2,24.1,0.0,0.262259,...,0.360169,0.046610,0.016949,0.0,0.000000,0.855932,0.050847,1.014907,0.496276,0
3,01611500132225,672.0,96.4,72.8,25.0,0.0,3.5,28.0,0.9,0.166358,...,0.161074,0.315436,0.020134,0.0,0.000000,0.912752,0.033557,0.829876,0.470174,0
5,01611680132746,40.0,95.0,76.3,0.0,0.0,29.6,67.9,0.0,0.551136,...,0.333333,0.166667,0.000000,0.0,0.000000,1.000000,0.000000,1.170732,0.545455,0


# Import School Information Raw Dataset


In [35]:
# import school data
df_school_data = rpkl( raw_pickle, "raw_school_data.pkl")
df_school_data.head()

ℹ️ 'cdscode' already exists — skipping creation

📁 Columns in raw_school_data.pkl:
['cdscode', 'ncesdist', 'ncesschool', 'statustype', 'county', 'district', 'school', 'street', 'streetabr', 'city', 'zip', 'state', 'mailstreet', 'mailstrabr', 'mailcity', 'mailzip', 'mailstate', 'phone', 'ext', 'faxnumber', 'website', 'opendate', 'closeddate', 'charter', 'charternum', 'fundingtype', 'doc', 'doctype', 'soc', 'soctype', 'edopscode', 'edopsname', 'eilcode', 'eilname', 'gsoffered', 'gsserved', 'virtual', 'magnet', 'yearroundyn', 'federaldfcdistrictid', 'latitude', 'longitude', 'admfname', 'admlname', 'lastupdate', 'multilingual']


,cdscode,ncesdist,ncesschool,statustype,county,district,school,street,streetabr,city,...,virtual,magnet,yearroundyn,federaldfcdistrictid,latitude,longitude,admfname,admlname,lastupdate,multilingual
59,01611190130229,0601770,00041,Active,Alameda,Alameda Unified,Alameda High,2200 Central Avenue,2200 Central Ave.,Alameda,...,N,N,N,No Data,37.764958,-122.24593,Angela,Barrett,2024-09-30,N
91,01611270130450,0601860,00059,Active,Alameda,Albany City Unified,Albany High,603 Key Route Boulevard,603 Key Route Blvd.,Albany,...,N,N,N,No Data,37.896661,-122.29257,Darren,McNally,2023-02-09,N
113,01611430131177,0604740,00432,Active,Alameda,Berkeley Unified,Berkeley High,1980 Allston Way,1980 Allston Way,Berkeley,...,N,N,N,No Data,37.868913,-122.27120,Juan,Raygoza,2020-08-13,Y
153,01611500132225,0607800,00742,Active,Alameda,Castro Valley Unified,Castro Valley High,19400 Santa Maria Avenue,19400 Santa Maria Ave.,Castro Valley,...,N,N,N,No Data,37.705184,-122.07847,Christopher,Fortenberry,2023-02-09,N
154,01611500133876,0607800,14010,Active,Alameda,Castro Valley Unified,Castro Valley Virtual Academy,18400 Clifton Way,18400 Clifton Way,Castro Valley,...,V,N,N,No Data,37.713501,-122.09222,Kenneth,Kahn,2023-02-09,N


In [36]:
df_school_data = df_school_data[['cdscode', 'county', 'district', 'school', 'latitude', 'longitude']]

df_school_data.head()

,cdscode,county,district,school,latitude,longitude
59,01611190130229,Alameda,Alameda Unified,Alameda High,37.764958,-122.24593
91,01611270130450,Alameda,Albany City Unified,Albany High,37.896661,-122.29257
113,01611430131177,Alameda,Berkeley Unified,Berkeley High,37.868913,-122.27120
153,01611500132225,Alameda,Castro Valley Unified,Castro Valley High,37.705184,-122.07847
154,01611500133876,Alameda,Castro Valley Unified,Castro Valley Virtual Academy,37.713501,-122.09222


## Merge school information with dataset


In [37]:
df = df.merge(
    df_school_data, 
    on = "cdscode",
    how="left"
)

df.head()

,cdscode,cohortstudents,graduation_rate,met_uccsu_grad_reqs_rate,seal_of_biliteracy_rate,still_enrolled_rate,chronicabsenteeismrate,unexcused_absences_percent,outofschool_suspension_absences_percent,percent__eligible_free_k12,...,pct_experienced,pct_second_year,grade_retention_ratio,pct_senior_cohort,low_grad_rate,county,district,school,latitude,longitude
0,01611190130229,394.0,92.4,73.9,22.8,1.0,12.7,23.5,0.5,0.172013,...,0.863158,0.031579,1.058568,0.498894,0,Alameda,Alameda Unified,Alameda High,37.764958,-122.24593
1,01611270130450,284.0,95.1,67.8,21.5,0.0,70.3,46.2,0.4,0.174389,...,0.894523,0.038462,0.996721,0.501264,0,Alameda,Albany City Unified,Albany High,37.896661,-122.29257
2,01611430131177,861.0,90.5,62.3,12.1,0.8,5.2,24.1,0.0,0.262259,...,0.855932,0.050847,1.014907,0.496276,0,Alameda,Berkeley Unified,Berkeley High,37.868913,-122.27120
3,01611500132225,672.0,96.4,72.8,25.0,0.0,3.5,28.0,0.9,0.166358,...,0.912752,0.033557,0.829876,0.470174,0,Alameda,Castro Valley Unified,Castro Valley High,37.705184,-122.07847
4,01611680132746,40.0,95.0,76.3,0.0,0.0,29.6,67.9,0.0,0.551136,...,1.000000,0.000000,1.170732,0.545455,0,Alameda,Emery Unified,Emery Secondary,37.835405,-122.28270


# Import Top Features


In [38]:
# import top features
top_features_path = Path("../models/top_features.pkl")
top_features = joblib.load(top_features_path)
print(top_features)

['still_enrolled_rate', 'chronicabsenteeismrate', 'unexcused_absences_percent', 'met_uccsu_grad_reqs_rate', 'percent__eligible_free_k12', 'frpm_count_k12', 'stu_tch_ratio', 'pct_experienced', 'cohortstudents', 'pct_senior_cohort', 'stu_adm_ratio', 'grade_retention_ratio', 'pct_bachelors_plus', 'stu_psv_ratio', 'pct_bachelors']


In [39]:
identifier_cols = ['cdscode', 'county', 'district', 'school', 'latitude', 'longitude']
outcome_cols = ["graduation_rate", "low_grad_rate"]
final_columns = identifier_cols + outcome_cols + top_features
print(final_columns)

['cdscode', 'county', 'district', 'school', 'latitude', 'longitude', 'graduation_rate', 'low_grad_rate', 'still_enrolled_rate', 'chronicabsenteeismrate', 'unexcused_absences_percent', 'met_uccsu_grad_reqs_rate', 'percent__eligible_free_k12', 'frpm_count_k12', 'stu_tch_ratio', 'pct_experienced', 'cohortstudents', 'pct_senior_cohort', 'stu_adm_ratio', 'grade_retention_ratio', 'pct_bachelors_plus', 'stu_psv_ratio', 'pct_bachelors']


In [40]:
df_final = df[final_columns].copy()

print(df_final.shape)
df_final.head()

(958, 23)


,cdscode,county,district,school,latitude,longitude,graduation_rate,low_grad_rate,still_enrolled_rate,chronicabsenteeismrate,...,frpm_count_k12,stu_tch_ratio,pct_experienced,cohortstudents,pct_senior_cohort,stu_adm_ratio,grade_retention_ratio,pct_bachelors_plus,stu_psv_ratio,pct_bachelors
0,01611190130229,Alameda,Alameda Unified,Alameda High,37.764958,-122.24593,92.4,0,1.0,12.7,...,327.0,23.3,0.863158,394.0,0.498894,452.0,1.058568,0.315789,361.6,0.126316
1,01611270130450,Alameda,Albany City Unified,Albany High,37.896661,-122.29257,95.1,0,0.0,70.3,...,307.0,22.0,0.894523,284.0,0.501264,414.0,0.996721,0.264133,274.7,0.166667
2,01611430131177,Alameda,Berkeley Unified,Berkeley High,37.868913,-122.27120,90.5,0,0.8,5.2,...,935.0,19.2,0.855932,861.0,0.496276,374.7,1.014907,0.148305,228.7,0.427966
3,01611500132225,Alameda,Castro Valley Unified,Castro Valley High,37.705184,-122.07847,96.4,0,0.0,3.5,...,491.0,22.6,0.912752,672.0,0.470174,396.9,0.829876,0.315436,178.2,0.187919
4,01611680132746,Alameda,Emery Unified,Emery Secondary,37.835405,-122.28270,95.0,0,0.0,29.6,...,102.0,11.6,1.000000,40.0,0.545455,176.0,1.170732,0.166667,274.7,0.333333


In [41]:
filename_pkl = "06_top15_features_w_ids_and_target.pkl"
filename_csv = "06_top15_features_w_ids_and_target.csv"

# save to pkl
df_final.to_pickle( data_folder / filename_pkl)

# save to csv 
df_final.to_csv(data_folder / filename_csv, index=False)

print(f"Saved: {filename}")

Saved: 06__top15_features_w_ids_and_target.pkl
